<a href="https://colab.research.google.com/github/azuzow/ooga/blob/master/Ooga.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#all libraries used
!apt install chromium-chromedriver
!pip install selenium 
!pip install fuzzywuzzy
import math
import random
import urllib.request
import requests
import ssl
import time
from googlesearch import search
import re
from bs4 import BeautifulSoup

from fuzzywuzzy import fuzz
from selenium import webdriver

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')


Reading package lists... Done
Building dependency tree       
Reading state information... Done
chromium-chromedriver is already the newest version (79.0.3945.79-0ubuntu0.18.04.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 7 not upgraded.


In [0]:
#gets all webpages for query
pagesWithoutRankings=[]
amzonLinks=[]
def getLinks(query='top 10 ranked gaming keyboards'):
  domainList = []
  searchList= []
  for count,i in enumerate (search(query, tld="com", num=200, start=0, stop=200, pause=2)):
      domain = i.split('//')[-1].split('/')[0]
      domain = domain.lstrip('www.')
      domain = "https://www." + domain
      #if a domain has already been seen dont add the link to the pages that will be scraped
      if domain not in domainList:
          domainList.append(domain)
          print(count)
          # print(i)
          searchList.append(i)
  # print(searchList)
  return searchList
def visible(element):
  if element.parent.name in ['style', 'script', '[document]', 'head', 'title']:
    return False
  elif re.match('<!--.*-->', str(element.encode('utf-8'))):
    return False
  return True

def getPages(urls):
  pages = []
  cc=1
  amazonLinks=set()
  #problems occur here, i think when try doesnt work the first time? im not sure
  junkWords=["These ","quite","to","too","like","This",'this','because',"it's"]
  # junkWords=[]

  for items in urls:
      try:
          html = requests.get(items,timeout=5).text
          soup = BeautifulSoup(html,'html.parser')

          data = soup.findAll(text=True)

          #finds all links then gets all links with amazon referenced 
          links = [a.get('href') for a in soup.find_all('a', href=True)]
          for link in links:
            if 'amazon' in link:
              if(link not in amazonLinks):
                amazonLinks.add(link)
                # print(link)
     
          visible_texts = filter(visible, data)
          temp=''
          for t in visible_texts:
            junkLine=False
            for w in junkWords: 
              if w in t:
                junkLine=True
                break
            if not junkLine:
              # print("==============================")
              # print(t)
              # print(t.strip())
              # print(type(t))
              temp =temp+ (t.strip()+"\n")
              # print(temp)
              # print("==============================")
          visible_texts=temp
          pages.append(visible_texts)
      except Exception as e:
          print('couldnt get page',items)
          print(e)
      #soup = BeautifulSoup(html)
      #data = soup.findAll(text=True)
      print(cc)
      cc+=1
  return pages,amazonLinks

  #how many pages have visible text to scrape 
  # print(len(pages))
#all pages with visible text from query
def findLists():
  pages,amazonLinks=getPages(getLinks())

  lists=[]

  regEx = "\d+[.][' '][a-zA-z].*"

  for page in pages:
    #come back and do something with page if no rankings are found
    lists.append(re.findall(regEx,page))
  # print("=================================== PRODUCT NAMES ===============================")
  # for list in lists:
  #   print(list)
  # print("=================================== PRODUCT NAMES ===============================")
  return lists,amazonLinks





In [0]:
 def listformat(lstoflst):
  #this section turns each item into a list [rank,name], so we have [[[rank, name][rank,name]][[rank,name][rank,name]]]
  newlist=[]
  for i in range(len(lstoflst)):
    templist=[]
    print("============================= PAGE",i+1,"=============================")
    for k in range(len(lstoflst[i])):
      print(lstoflst[i][k])
      rank=lstoflst[i][k][0:3]
      name=lstoflst[i][k][3:]
      templist.append([rank,name])
    newlist.append(templist)
  return(newlist)

      



def listcleanblanks(lst):
  for i in (lst[:]):
    if i==[]:
      lst.remove(i)
  return(lst)

def removespaceandcombo(lst):
  for i in range(len(lst)):
    for k in range(len(lst[i])):
      
      if lst[i][k][1][0] == ' ':
        lst[i][k][1]=lst[i][k][1][1:]
      if lst[i][k][0][-1]==' ':
        lst[i][k][0]=lst[i][k][0][:-1]
  newlist=[]
  for i in range(len(lst)):
    for k in range(len(lst[i])):
      newlist.append(lst[i][k])

  return(newlist)

def getAmazonASIN(amazonLinks):
  productsASIN=set()
  urls=set()
  for link in amazonLinks:
      try:
        headers = {
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
        "Accept-Encoding": "gzip, deflate, sdch, br",
        "Accept-Language": "en-US,en;q=0.8",
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36"}
        regEx= '[A-Z0-9]{10}[?]'
        request = requests.get(link, headers=headers, allow_redirects=True)
        url = request.url
        ASIN = re.search(regEx,url)
        if(ASIN!=None):
          urls.add(url)
          ASIN = ASIN.group(0)        
          ASIN=ASIN[:10]
          productsASIN.add(ASIN)
      except Exception as e:
        print(e) 
  return productsASIN    


lst,amazonLinks=findLists()
lst=listformat(lst)
lst=listcleanblanks(lst)
#lst=removespaceandcombo(lst)
amazonASIN= getAmazonASIN(amazonLinks)
# print(*lst,sep='/n')
# print('-----------------------------')
numoflists=len(lst)



0
7
8
9
10
11
15
16
17
18
19
20
21
22
24
25
27
28
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
1
2
3
4
5
6
7
8
9
couldnt get page https://www.cnet.com/news/best-gaming-keyboards-under-200-2020/?sa=X&ved=2ahUKEwjAhambmv_mAhVzCjQIHetnBS0QFjAMegQIQxAB
HTTPSConnectionPool(host='www.cnet.com', port=443): Read timed out. (read timeout=5)
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
============================= PAGE 1 =============================
1. SteelSeries Apex Pro
2. Razer Huntsman Elite
3. Roccat Vulcan 120 Aimo
4. Logitech G513
5. Alienware Pro Gaming Keyboard AW7

In [0]:
def createASINPairs(ASIN):
  trueNames=dict()
  wd = webdriver.Chrome('chromedriver',options=options)
  for id in ASIN:
    try:
      query = 'http://www.amazon.com/s?k=ASIN&ref=nb_sb_noss'
      query= query.replace('ASIN',str(id))
      wd.get(query)
      trueNames[wd.find_element_by_class_name('s-image').get_attribute("alt")]=id
    except Exception as e:
      print(e,query)
  for k, v in trueNames.items():
    print (k, '-->', v)
  return trueNames
ooga=createASINPairs(amazonASIN)



Message: no such element: Unable to locate element: {"method":"css selector","selector":".s-image"}
  (Session info: headless chrome=79.0.3945.79)
 http://www.amazon.com/s?k=B07DHXC3P1&ref=nb_sb_noss
Message: no such element: Unable to locate element: {"method":"css selector","selector":".s-image"}
  (Session info: headless chrome=79.0.3945.79)
 http://www.amazon.com/s?k=B07KMLLPPW&ref=nb_sb_noss
Message: no such element: Unable to locate element: {"method":"css selector","selector":".s-image"}
  (Session info: headless chrome=79.0.3945.79)
 http://www.amazon.com/s?k=B01H036P6W&ref=nb_sb_noss
Message: no such element: Unable to locate element: {"method":"css selector","selector":".s-image"}
  (Session info: headless chrome=79.0.3945.79)
 http://www.amazon.com/s?k=B01K32MU2A&ref=nb_sb_noss
Message: no such element: Unable to locate element: {"method":"css selector","selector":".s-image"}
  (Session info: headless chrome=79.0.3945.79)
 http://www.amazon.com/s?k=B00MTIBDEI&ref=nb_sb_noss


In [0]:
def similar(product, url):
  Token_Set_Ratio = fuzz.token_set_ratio(product.lower(),url.lower())
  #print(Token_Set_Ratio)
  return Token_Set_Ratio

def compareNames(lst,nameDict):
  rankings=[]
  for site in lst:
    for i in range(len(site)):
      for name in nameDict:
        if similar(site[i][1],name) >= 80:
          rankings.append([site[i][1],similar(site[i][1],name),name])
 # print(*rankings,sep='\n')
  rankings = sorted(rankings, key = lambda x: (x[0], x[1]))
  rankings.reverse()
  print(*rankings,sep='\n')
  newlist=[]
  for site in lst:
    for i in range(len(site)):
      for k in range(len(rankings)):
        if site[i][1] == rankings[k][0]:
          newlist.append([site[i][0],rankings[k][0],rankings[k][2]])
          break;
  #print(*newlist,sep='\n')
  return newlist
def connectasin(lst,namedict):
  newlist=[]
  for item in lst:
    newlist.append([item[0],item[2],namedict[item[2]]])
  print(newlist)
  return newlist
one=compareNames(lst,ooga)
two=connectasin(one,ooga)  
print(*two,sep='\n')
# lst=compareNames(lst,ooga)
# lst=connectasin(lst,ooga)

['XPG Summoner', 100, 'XPG Summoner RGB Mechanical Gaming Keyboard - Cherry Speed Silver Switch - Detachable Magnetic Wrist Rest - USB Passthrough & Media Controls (SUMMONER4C-BKCWW)']
['Wooting One & Wooting Two', 85, 'WOOTING ONE Linea\u200d\u200d\u200dr55 Red\u200d\u200d Analog RGB TKL Mechanical Gaming Keyboard: Pressure Sensitive, Adjustable Actuation Point, RGB Backlight, On-Board Memory, No Tactile and Audible Feedback.']
['Which Gaming Keyboard is Best for First Person Shooters?', 81, 'Pictek Gaming Keyboard']
['VAVA Mechanical Keyboard', 88, 'RK61 60% RGB Mechanical Gaming Keyboard Small Compact 61 Keys, Wired/Wireless Bluetooth Mini Portable Keyboard Gaming/Office for iOS Android Windows and Mac with Programmer Red Switch - White']
['VAVA Mechanical Keyboard', 88, 'Logitech G413 Backlit Mechanical Gaming Keyboard with USB Passthrough – Carbon']
['VAVA Mechanical Keyboard', 88, 'CORSAIR K70 RGB RAPIDFIRE Mechanical Gaming Keyboard - USB Passthrough & Media Controls - Fastest &

In [0]:
#recieving list of lists, each element of the form ['rank from a site', 'name', 'ASIN']
def organizetriplelist(lst):
  for i in range(len(lst)):
    loc=lst[i][0].find('.')
    lst[i][0]=lst[i][0][0:loc]
  for i in range(len(lst)):
    lst[i].reverse()
    lst[i][-1]=int(lst[i][-1])
    
  return lst



def listcompression(lst):
  asin=[]
  newlist=[]
  for i in range(len(lst)):
    asin.append(lst[i][0])
  asin=set(asin)
  for i in asin:
     newlist.append([i])
  for i in range(len(newlist)):
    for k in range(len(lst)):
      if lst[k][0]==newlist[i][0]:
        if len(newlist[i])==1:
          newlist[i].append(lst[k][1])
        newlist[i].append(lst[k][2])

  for i in range(len(newlist)):
    newlist[i].append((sum(newlist[i][2:]))/(len(newlist[i])-2))
    newlist[i].append(len(newlist[i])-3)
  return(newlist)

def addnumoflists(lst,num):
  for i in range(len(lst)):
    lst[i].append(num)
  return lst


 #five = [asin,name,...,ave,freq,#oflists]
def addscore(lst):
  for i in range(len(lst)):
    score = lst[i][-3] + -20*(math.log((2/3)+((lst[i][-2])/(lst[i][-1]))))
    lst[i].append(score)
  return lst


def finalsortbyscore(lst):
  lst.sort(key = lambda lst: lst[-1])
  return lst

print('===========================================')
three=organizetriplelist(two)
# print(*three,sep='\n')
# print('===========================================')
four=listcompression(three)
# print(*four,sep='\n')
# print('===========================================')
five=addnumoflists(four,numoflists)
# print(*five,sep='\n')
# print('===========================================')
six=addscore(five)
#print(*six,sep='\n')

seven = finalsortbyscore(six)
print('===========================================')
print(*seven,sep='\n')






# lst=testing()
# print('========================')
# lst=listcompression(lst)
# print(*lst,sep='\n')

['B01M4LIKLI', 'Corsair K55 RGB Gaming Keyboard - Quiet & Satisfying LED Backlit Keys - Media Controls - Wrist Rest Included – Onboard Macro Recording', 1, 5, 5, 5, 5, 5, 5, 1, 1, 2, 3.5, 10, 41, 5.37370968154646]
['B077YDJWZR', 'HyperX Alloy Elite RGB - Mechanical Gaming Keyboard - Software-Controlled Light & Macro Customization - Wrist Rest - Media Controls - Linear & Quiet - Cherry MX Red - RGB LED Backlit (HX-KB2RD2-US/R1)', 2, 5, 6, 3, 7, 3, 5, 6, 6, 3, 4.6, 10, 41, 6.47370968154646]
['B019O9BLVY', 'Redragon K552-RGB Mechanical Gaming Keyboard Compact 87 Key Mechanical Computer Keyboard KUMARA USB Wired Cherry MX Blue Equivalent Switches for Windows PC Gamers (Black RGB Backlit)', 12, 10, 2, 4, 4, 4, 4, 3, 3, 1, 10, 5.181818181818182, 11, 41, 6.526862722001531]
['B07PBC15N8', 'RK61 60% RGB Mechanical Gaming Keyboard Small Compact 61 Keys, Wired/Wireless Bluetooth Mini Portable Keyboard Gaming/Office for iOS Android Windows and Mac with Programmer Red Switch - White', 5, 1, 4, 1, 4